In [17]:
import pandas as pd
from IPython.display import display, HTML, Image

# Crear un DataFrame con una columna que contiene URLs de imágenes
data = {'Nombre': ['Hunger Games', 'Harry Potter 2'],
        'Autor': ['Suzanne Collins', 'J.K. Rowling'],
        'URL': ['https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1586722975i/2767052.jpg', 
                'https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1546910265i/2.jpg']}
df = pd.DataFrame(data)

# Función para mostrar solo el nombre de la imagen seguido de la imagen
def mostrar_nombre_con_imagen(df_row):
    nombre_imagen = df_row['Nombre'].upper()
    autor = df_row['Autor']
    imagen_url = df_row['URL']

    html_output = f"<h3>{nombre_imagen}<br>{autor}</h3>"
    html_output += f"<img src='{imagen_url}' width='100' height='100'>"

    display(HTML(html_output))

# Itera sobre las filas del DataFrame y muestra el nombre con su imagen
for index, row in df.iterrows():
    mostrar_nombre_con_imagen(row)

In [2]:
df

,Nombre,URL
0,Imagen 1,https://images-na.ssl-images-amazon.com/images...
1,Imagen 2,https://images-na.ssl-images-amazon.com/images...


In [5]:
df.URL[0].apply(mostrar_imagen)

AttributeError: 'str' object has no attribute 'apply'

In [ ]:
# Aplicar filtros
filtered_df = df[
    (df['author'].str.contains(author_filter, case=False)) &
    (df['genre'].apply(lambda x: any(item in selected_genres for item in x.values()))) &
    (df['rating'] >= min_rating) &
    (df['rating'] <= max_rating) &
    (df['publication_date'] >= date)
]

# Mostrar los datos filtrados
for index, row in filtered_df.iterrows():
    st.write(f"**Title:** {row['title']}")
    st.write(f"**Author:** {row['author']}")
    st.write(f"**Genre:** {', '.join(row['genre'].values())}")
    st.write(f"**Rating:** {row['rating']}")
    st.write(f"**Number of pages:** {row['num_page']}")
    st.image(row['cover'], caption='Cover', use_column_width=True)
    st.write('---')

In [ ]:

# Title for the side bar
st.sidebar.header('Filters')

# Filter for the author
unique_authors = df['author'].unique()
unique_authors = ['Choose an option'] + list(unique_authors)
selected_author = st.sidebar.selectbox('Author', unique_authors)
if selected_author != 'Choose an option':
    filtered_df = df[df['author'] == selected_author]
    st.write(filtered_df)
    st.write(f"Author selected: {selected_author}")
else:
    st.write("No author selected")

# Filter for the genre
df['genre'] = df['genre'].apply(lambda x: ast.literal_eval(x))
unique_genres = set()
for value in df['genre']:
    unique_genres.update(value.values())
selected_genres = st.sidebar.multiselect('Genre', list(unique_genres))
filtered_df = df[df['genre'].apply(lambda x: any(item in selected_genres for item in x.values()))]
if selected_genres:
    st.write(filtered_df)
else:
    st.write("No genres selected")


# Filter for the publication year
years = sorted(df['publication_date'].dt.year.unique())
years.insert(0, '')
selected_year = st.sidebar.select_slider('Publication year', options=years)
if selected_year != '':
    filtered_df = df[df['publication_date'].dt.year == selected_year]
    st.write(filtered_df)
else:
    st.write("No year selected")

# Filtro para la calificación
min_rating, max_rating = st.sidebar.slider('Rating', 0.0, 5.0, (0.0, 5.0))
if min_rating != 0.0 or max_rating != 5.0:
    filtered_df = df[(df['rating'] >= min_rating) & (df['rating'] <= max_rating)]
    if not filtered_df.empty:
        st.write(filtered_df)
    else:
        st.write("No matching data")
else:
    st.write("No rating filter applied")

# Filtro para el número de ratings
min_num_ratings = df['rating_count'].min()
max_num_ratings = df['rating_count'].max()

# Filtro para el número de ratings con valores por defecto que abarquen todo el rango
selected_min, selected_max = st.sidebar.slider('Number of ratings', min_num_ratings, max_num_ratings, (min_num_ratings, max_num_ratings))
if (selected_min, selected_max) != (min_num_ratings, max_num_ratings):
    filtered_df = df[(df['rating_count'] >= selected_min) & (df['rating_count'] <= selected_max)]
    if not filtered_df.empty:
        st.write(filtered_df)
    else:
        st.write("No matching data")
else:
    st.write("No number of ratings filter applied")

# Filtro para el número de paginas
min_num_pages = df['num_page'].min()
max_num_pages = df['num_page'].max()
selected_min, selected_max = st.sidebar.slider('Number of pages', min_num_pages, max_num_pages, (min_num_pages, max_num_pages))
if (selected_min, selected_max) != (min_num_pages, max_num_pages):
    filtered_df = df[(df['num_page'] >= selected_min) & (df['num_page'] <= selected_max)]
    if not filtered_df.empty:
        st.write(filtered_df)
    else:
        st.write("No matching data")
else:
    st.write("No number of pages filter applied")

In [ ]:
# Mostrar los resultados filtrados
if not base_df.empty:
    st.write(base_df)
else:
    st.write("No matching data")

In [ ]:
if filters_applied and not base_df.empty:
    num_columns = 3  # Número de columnas para mostrar los resultados

    # Divide el espacio de la página en 'num_columns' columnas
    columns = st.columns(num_columns)

    for index, row in base_df.iterrows():
        with columns[index % num_columns]:  # Distribuye los resultados en las columnas
            st.markdown(f"**Title:** {row['title']}")
            st.markdown(f"**Author:** {row['author']}")
            st.markdown(f'**Genre:** {", ".join(row["genre"].values())}')
            rating_with_emoji = map_rating_to_emoji(row['rating'])
            st.markdown(f"**Rating:** {rating_with_emoji}")
            st.markdown(f'**Number of ratings:** {row["rating_count"]}')
            st.markdown(f'**Number of reviews:** {row["review_count"]}')
            st.markdown(f'**Publication date:** {row["publication_date"].strftime("%d/%m/%Y")}')
            st.markdown(f'**Number of pages:** {row["num_page"]}')

            cover_url = row['cover']  # Asigna el valor de la columna 'cover' que contiene la URL de la imagen
            if cover_url:
                st.image(cover_url, caption='Cover', width=100)
            else:
                st.write("No cover available")

            # Salto de línea entre cada libro
            st.markdown("---")
elif not filters_applied:
    st.write("Select filters to display results 📚")
else:
    st.write("No matching data")